## Packages to Create Maps

In [3]:
import numpy as np #https://numpy.org/
import pandas as pd #https://pandas.pydata.org
import geopandas as gpd #https://geopandas.org/en/stable/getting_started/introduction.html
from geopandas.tools import geocode
import geopy
import requests #https://docs.python-requests.org/en/latest/index.html
import mapclassify #https://pysal.org/mapclassify/
import folium #https://python-visualization.github.io/folium/latest/#
import matplotlib #https://matplotlib.org
from shapely.geometry import Point #https://shapely.readthedocs.io/en/stable/geometry.html

## Ohio State Shape Files

In [5]:
#Ohio
Ohio_Shape_File = gpd.read_file('tl_2024_39_tract')
Ohio_Shape_File['GEOID'] = Ohio_Shape_File.GEOID.astype(int) #need this to be an integer so that you can merge with other files
Ohio_Shape_File.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 3168 entries, 0 to 3167
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   STATEFP   3168 non-null   object  
 1   COUNTYFP  3168 non-null   object  
 2   TRACTCE   3168 non-null   object  
 3   GEOID     3168 non-null   int64   
 4   GEOIDFQ   3168 non-null   object  
 5   NAME      3168 non-null   object  
 6   NAMELSAD  3168 non-null   object  
 7   MTFCC     3168 non-null   object  
 8   FUNCSTAT  3168 non-null   object  
 9   ALAND     3168 non-null   int64   
 10  AWATER    3168 non-null   int64   
 11  INTPTLAT  3168 non-null   object  
 12  INTPTLON  3168 non-null   object  
 13  geometry  3168 non-null   geometry
dtypes: geometry(1), int64(3), object(10)
memory usage: 346.6+ KB


## Loading Ohio Census File

In [7]:
Census_File = pd.read_csv('CensusFlatFile2024.csv')
Census_File['GEOID'] = Census_File['SCCT']
Census_File.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3168 entries, 0 to 3167
Data columns (total 17 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   MSA Code                         3168 non-null   int64  
 1   State FIP                        3168 non-null   int64  
 2   County FIP                       3168 non-null   int64  
 3   Census Tract                     3168 non-null   int64  
 4   SCCT                             3168 non-null   int64  
 5   MFI                              3168 non-null   int64  
 6   MHI                              3168 non-null   int64  
 7   LMI%                             3168 non-null   float64
 8   Census Tract Income Indicator    3168 non-null   object 
 9   Total persons                    3168 non-null   int64  
 10  Total families                   3168 non-null   int64  
 11  Total households                 3168 non-null   int64  
 12  HMM%                

## Merging DataFrames and Cleaning

In [9]:
#Merging ShapeOhio_Shape_File_Shape_Fileile to Census File
Df = pd.merge(Ohio_Shape_File, Census_File, on = 'GEOID', how = 'left')
Df['GEOID'] = Df.GEOID.astype(str)
Df = Df.drop(['SCCT'], axis = 1)

In [10]:
#Removing Water Census Tracts

Df['CT'] = Df['GEOID'].str[-6:]
Df = Df.loc[Df['CT'] != '990000']
Df = Df.loc[Df['CT'] != '990100']
Df = Df.loc[Df['CT'] != '990200']
Df = Df.drop(['CT'], axis = 1)

In [11]:
Df = Df.loc[Df['MSA Name'] == 'Cincinnati, OH-KY-IN']

## Map Creation

In [13]:
m = Df.explore(
    column = 'County',
    categorical = True,
    #cmap = ['#297dc1','#f98909'], #Changes the colors of your column variable
    scheme = 'naturalbreaks',
    legend = True,
    legend_kwds = dict(caption = 'Counties List', colorbar = False),
    tooltip = False, #hides tooltip
    popup = ['MSA Name',
            'County',
            'Census Tract Income Indicator',
            'Census Tract Minority Indicator',
            'GEOID'], #Shows all of the information when clicked on
    popup_kwds = dict(aliases = ['MSA Name', 
                                  'County Name',
                                  'Income Indicator',
                                  'Minority Indicator', 
                                  'Geographic ID']), #update for readability
    name = 'Counties List',
    style_kwds = dict(color='black', fillOpacity = .40)
)

folium.TileLayer("CartoDB Positron", show = False).add_to(m)
folium.LayerControl().add_to(m)
folium.FitOverlays().add_to(m)

m

In [14]:
m.save("Cincinnati, OH.html")